In [0]:
# Sean Wendlandt
# Lab 6 5/4/23
sc = spark.sparkContext
import pyspark.sql.functions as f

In [0]:
# Read in master file
masterFile = "dbfs:///FileStore/tables/Master-2.csv"
masterDF = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("mode", "dropMalformed").load(masterFile)
#masterDF.head()

In [0]:
# Read in teams file
teamsFile = "dbfs:///FileStore/tables/Teams.csv"
teamsDF = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("mode", "dropMalformed").load(teamsFile)
# teamsDF.head()

In [0]:
# Read in allstar file
allstarFile = "dbfs:///FileStore/tables/AllstarFull.csv"
allstarDF = spark.read.format("csv").option("header", True).option("ignoreLeadingWhiteSpace", True).option("mode", "dropMalformed").load(allstarFile)
# allstarDF.head()

In [0]:
# Selecting the specific feilds we are interested in: nameFirst, nameLast, teamID, playerID
masterDF = masterDF.select(f.col("playerID"), f.col("nameFirst"), f.col("nameLast"))
# masterDF.show()
teamsDF = teamsDF.select(f.col("teamID"), f.col("name").alias("teamName"))
# teamsDF.show()
allstarDF = allstarDF.select(f.col("playerID"), f.col("teamID"))
# allstarDF.show()

In [0]:
# Join feilds into single df, distinct to remove duplicates
joinedDF = allstarDF.join(masterDF, "playerID").join(teamsDF, "teamID").distinct()
joinedDF.show()


+------+---------+---------+---------+--------------------+
|teamID| playerID|nameFirst| nameLast|            teamName|
+------+---------+---------+---------+--------------------+
|   BAL| aasedo01|      Don|     Aase|   Baltimore Orioles|
|   PHI|allendi01|     Dick|    Allen|Philadelphia Blue...|
|   NY1|adamsac01|      Ace|    Adams|    New York Gothams|
|   PHI|abreubo01|    Bobby|    Abreu|Philadelphia Blue...|
|   NYN|alfoned01|  Edgardo|  Alfonzo|       New York Mets|
|   NY1|adamsac01|      Ace|    Adams|     New York Giants|
|   DET|aguirha01|     Hank|  Aguirre|      Detroit Tigers|
|   ML1|adcocjo01|      Joe|   Adcock|    Milwaukee Braves|
|   CHA|allendi01|     Dick|    Allen|   Chicago White Sox|
|   PHI|allendi01|     Dick|    Allen|Philadelphia Phil...|
|   MIN|aguilri01|     Rick| Aguilera|     Minnesota Twins|
|   ML1|aaronha01|     Hank|    Aaron|    Milwaukee Braves|
|   CHA| ageeto01|   Tommie|     Agee|   Chicago White Sox|
|   DET|alexado01|    Doyle|Alexander|  

In [0]:
# Saving as Parquet
parFilename = "dbfs:///FileStore/tables/allstars/"
dbutils.fs.rm(parFilename, True)
joinedDF.write.format("parquet").mode("overwrite").save(parFilename)

In [0]:
# Read parquet file
newDF = spark.read.format("parquet").load(parFilename)
newDF.show()

+------+---------+------------+----------+--------------------+
|teamID| playerID|   nameFirst|  nameLast|            teamName|
+------+---------+------------+----------+--------------------+
|   BAL| aasedo01|         Don|      Aase|   Baltimore Orioles|
|   BAL|brandja01|      Jackie|    Brandt|   Baltimore Orioles|
|   BRO|camildo01|       Dolph|   Camilli|Brooklyn Bridegrooms|
|   HOU|cedence01|       Cesar|    Cedeno|  Houston Colt .45's|
|   BSN|fettelo01|         Lou|     Fette|         Boston Bees|
|   DET|fieldce01|       Cecil|   Fielder|      Detroit Tigers|
|   CLE|frymatr01|      Travis|    Fryman|      Cleveland Naps|
|   BOS|hurstbr01|       Bruce|     Hurst|      Boston Red Sox|
|   NY1|jansela01|       Larry|    Jansen|    New York Gothams|
|   NYA|lazzeto01|        Tony|   Lazzeri|    New York Yankees|
|   CHA|mcdowja01|        Jack|  McDowell|   Chicago White Sox|
|   SLN|mizelvi01|Vinegar Bend|    Mizell| St. Louis Perfectos|
|   CLE|trillma01|       Manny|    Trill

In [0]:
# query the dataframe and show
newDF.filter(newDF["teamName"] == "Colorado Rockies").agg(f.count("playerID").alias("CR Allstar Total")).show()
newDF.filter(newDF["teamName"] == "Colorado Rockies").select(f.col("nameFirst"), f.col("nameLast")).show(newDF.count())

+----------------+
|CR Allstar Total|
+----------------+
|              24|
+----------------+

+---------+----------+
|nameFirst|  nameLast|
+---------+----------+
|  Jeffrey|  Hammonds|
|  Preston|    Wilson|
|     Matt|  Holliday|
|     Brad|     Hawpe|
|    Nolan|   Arenado|
|   Andres| Galarraga|
|    Larry|    Walker|
|  Charlie|  Blackmon|
|     Todd|    Helton|
|    Aaron|      Cook|
|   Ubaldo|   Jimenez|
|    Shawn|    Chacon|
|    Jason|   Marquis|
|    Ellis|     Burks|
|     Jeff|   Cirillo|
|     Troy|Tulowitzki|
|    Vinny|  Castilla|
|     Mike|   Hampton|
|    Brian|   Fuentes|
|  Michael|   Cuddyer|
|       DJ|  LeMahieu|
|   Carlos|  Gonzalez|
|     Eric|     Young|
|    Dante|  Bichette|
+---------+----------+

